In [3]:
# Import the necessary packages
import numpy as np
import cv2

def translate(image, x, y):
	# Define the translation matrix and perform the translation
	M = np.float32([[1, 0, x], [0, 1, y]])
	shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

	# Return the translated image
	return shifted

def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

def rotate(image, angle, center = None, scale = 1.0):
	# Grab the dimensions of the image
	(h, w) = image.shape[:2]

	# If the center is None, initialize it as the center of
	# the image
	if center is None:
		center = (w / 2, h / 2)

	# Perform the rotation
	M = cv2.getRotationMatrix2D(center, angle, scale)
	rotated = cv2.warpAffine(image, M, (w, h))

	# Return the rotated image
	return rotated

def resize(image, width = None, height = None, inter = cv2.INTER_AREA):
	# initialize the dimensions of the image to be resized and
	# grab the image size
	dim = None
	(h, w) = image.shape[:2]

	# if both the width and height are None, then return the
	# original image
	if width is None and height is None:
		return image

	# check to see if the width is None
	if width is None:
		# calculate the ratio of the height and construct the
		# dimensions
		r = height / float(h)
		dim = (int(w * r), height)

	# otherwise, the height is None
	else:
		# calculate the ratio of the width and construct the
		# dimensions
		r = width / float(w)
		dim = (width, int(h * r))

	# resize the image
	resized = cv2.resize(image, dim, interpolation = inter)

	# return the resized image
	return resized

In [4]:

import glob
import numpy as np


def uang_matching():
    # load templatel
    template_data = []
    template_files = glob.glob('template/*/v2/*.jpg', recursive=True)
    print("template loaded:", template_files)
    # prepare template
    for template_file in template_files:
        tmp = cv2.imread(template_file)
        tmp = resize(tmp, width=int(tmp.shape[1]*0.5))  # scalling
        tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)  # grayscale
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        tmp = cv2.filter2D(tmp, -1, kernel) #sharpening
        tmp = cv2.blur(tmp, (3, 3))  # smoothing
        tmp = cv2.Canny(tmp, 50, 200)  # Edge with Canny
        split_path = template_file.replace('template\\', '').replace('v2\\', '')
        nominal = split_path.split('\\')[0]
        template_data.append({"glob":tmp, "nominal":nominal})
    

    # template matching
    for image_glob in glob.glob('test/*.jpg'):
        for template in template_data:
            image_test = cv2.imread(image_glob)
            (tmp_height, tmp_width) = template['glob'].shape[:2]
            cv2.imshow("Template", template['glob'])

            image_test_p = cv2.cvtColor(image_test, cv2.COLOR_BGR2GRAY)
            cv2.imshow("Step: Grayscal", image_test_p)

            image_test_p = cv2.Canny(image_test_p, 50, 200)
            cv2.imshow("Step: edge with canny", image_test_p)

            found = None
            thershold = 0.4
            for scale in np.linspace(0.2, 1.0, 20)[::-1]:
                # scalling uang
                resized = resize(
                    image_test_p, width=int(image_test_p.shape[1] * scale))
                r = image_test_p.shape[1] / float(resized.shape[1])
                cv2.imshow("Step: rescale", resized)
                if resized.shape[0] < tmp_height or resized.shape[1] < tmp_width:
                    break

                # template matching
                result = cv2.matchTemplate(resized, template['glob'], cv2.TM_CCOEFF_NORMED)
                (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
                if found is None or maxVal > found[0]:
                    found = (maxVal, maxLoc, r)
                    if maxVal >= thershold:
                        print("money:", template['nominal'], "detected")
            if found is not None:
                (maxVal, maxLoc, r) = found
                (startX, startY) = (int(maxLoc[0]*r), int(maxLoc[1] * r))
                (endX, endY) = (
                    int((maxLoc[0] + tmp_width) * r), int((maxLoc[1] + tmp_height) * r))
                if maxVal <= thershold:
                    print(maxVal)
                    print(image_test.shape)
                    cv2.rectangle(image_test, (startX, startY),
                                  (endX, endY), (0, 0, 255), 2)
                cv2.imshow("Result", image_test)

            cv2.waitKey(0)


if __name__ == "__main__":
    uang_matching()
    
    cv2.waitKey(10)
    cv2.destroyAllWindows()

template loaded: ['template\\1000\\v2\\face.jpg', 'template\\1000\\v2\\nominal.jpg', 'template\\2000\\v2\\face.jpg', 'template\\2000\\v2\\nominal.jpg']
0.14670994877815247
(780, 1038, 3)
0.16931124031543732
(780, 1038, 3)
0.12869009375572205
(780, 1038, 3)
0.11718671023845673
(780, 1038, 3)
0.1639413684606552
(646, 1038, 3)
0.12998159229755402
(646, 1038, 3)
0.13083139061927795
(646, 1038, 3)
0.20497584342956543
(646, 1038, 3)
0.0936189517378807
(151, 334, 3)
0.07946579903364182
(151, 334, 3)
0.10604465007781982
(151, 334, 3)
0.27934491634368896
(151, 334, 3)
0.1821662336587906
(1600, 1200, 3)
0.17270885407924652
(1600, 1200, 3)
0.16731911897659302
(1600, 1200, 3)
0.13027885556221008
(1600, 1200, 3)
0.14357180893421173
(1600, 1200, 3)
0.1793576329946518
(1600, 1200, 3)
0.14018948376178741
(1600, 1200, 3)
0.15007425844669342
(1600, 1200, 3)
